# 異常偵測 - mustd

In [107]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json, datetime
from IPython.core.display import HTML
from IPython.display import display
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import sqlite3, requests
from datetime import datetime, timedelta


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
class DataAgent(object):
    """This class implement data query funciton
    functinos:
        gethvdata - 查詢健康值數據
        getcontroldata - 查詢控制器數據 
        
    """ 
    _defaults = {
        'PHMDB_PATH' : '/data/sqlite/phm_db/im_phm.db',
        'hv_cols' : ['hv_clamp', 'hv_eject', 'hv_inject', 'hv_temp', 'isdefect_clamp','isdefect_eject', 'isdefect_inject', 'isdefect_temp', 'moldidx', 'timestamp'],
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"
        
    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides    
        
        
    def gethvdata(self, ts1, ts2, mid):
        """
        從sqlite db抓設備健康值數據
        Parameters:
            ts1 - 開始時間(timestamp)
            ts2 - 結束時間(timestamp)
            mid - 設備ID(string)
        Returns:
            設備健康值數據(Dataframe)
        """
        conn = sqlite3.connect(self.PHMDB_PATH)
        cursor = conn.cursor()
        cursor.execute("PRAGMA table_info(healthvalue);")
        tbinfo = pd.DataFrame(cursor.fetchall()) 
        cursor.execute(f"select * from healthvalue where timestamp >= {ts1} and timestamp <= {ts2} and mid='{mid}'")
        data = pd.DataFrame(cursor.fetchall()) 
        data.columns= tbinfo[1].values  
        conn.close() 
        return data[self.hv_cols]

    def getcontroldata(self, ts1, ts2, mid, data_type):
        """
        從solr db抓控制器原始數據
        Parameters:
            ts1 - 開始時間(timestamp)
            ts2 - 結束時間(timestamp)
            mid - 設備ID(string) [A01, A03, A05, A06]
            data_type - 數據類型 (string) [spc, alarmrecord, historyrecord, machinestatus]
        Returns:
            Dataframe, 設備健康值數據
        """        
        QueryData_server_ip='http://10.160.29.112:8983/solr/{}/select?q=machine_id:{} AND timestamp:[{} TO {}]&rows=9999&sort=timestamp asc'.format(data_type,mid,ts1,ts2)
        r = requests.get(QueryData_server_ip)
        df = pd.DataFrame.from_dict(r.json()['response']['docs'], orient='columns')
        if df.shape[0]>0:
            df.drop(df.columns[df.columns.str.contains('_iso')],axis=1,inplace=True)
            df.drop(['id','machine_id','source_path','_version_'],axis=1,inplace=True)
            df = df.apply(pd.to_numeric, errors="ignore")
        try:
            df.drop_duplicates(subset="SPC_0",keep='first',inplace=True)
        except:
            pass
        df.reset_index(drop=True,inplace=True)
        df.sort_values(by='timestamp',inplace=True)
        df['dt'] = df['timestamp'].map(lambda x: datetime.fromtimestamp(x))
        return df
    
    

**計算每一個參數的mean, std**

In [110]:

dataAgent = DataAgent(PHMDB_PATH='db/im_phm.db')
strts = datetime(2019,9,25,11,0,0).timestamp()
endts = datetime(2019,9,25,13,0,0).timestamp()
mid='A01'

#抓控制器數據 ex 兩小時前的訓練資料
df_ctr = dataAgent.getcontroldata(strts, endts, mid, 'spc')

#Training的過程會根據報警的情況篩選出適當的數據
df_ctr.drop_duplicates(subset="SPC_0",keep='first',inplace=True)
df_ctr.reset_index(drop=True,inplace=True)
df_ctr.sort_values(by='timestamp',inplace=True)
df_ctr = df_ctr[-260:]

#取出SPC參數
d=df_ctr[['SPC_4','SPC_5','SPC_10','SPC_11','SPC_7','SPC_6','SPC_2','SPC_39','SPC_40','SPC_55','SPC_9',
                 'SPC_12','SPC_13','SPC_14','SPC_15','SPC_16','SPC_17','SPC_18','SPC_19','SPC_20','SPC_21','SPC_22','SPC_24','SPC_25','SPC_26','SPC_27','SPC_28',
                 'SPC_3','SPC_8','SPC_23','SPC_56','SPC_57',
                 'SPC_29','SPC_30','SPC_31','SPC_32','SPC_33','SPC_34','SPC_35','SPC_36','SPC_37']]

#算mean, std
bound = d.describe().loc[['mean','std']].T
bound.reset_index(drop=False,inplace=True)
bound.columns = ['spc_name','mean','std']
#output

**Inference Pass/Fail**

In [113]:

dataAgent = DataAgent(PHMDB_PATH='db/im_phm.db')
strts = datetime(2019,9,25,15,0,0).timestamp()
endts = datetime(2019,9,25,15,30,0).timestamp()
mid='A01'

#抓控制器數據 當下的30分鐘
df_ctr = dataAgent.getcontroldata(strts, endts, mid, 'spc')

#計算上下限
bound['upper'] = round(bound['mean']+6*bound['std'],2)
bound['lower'] = round(bound['mean']-6*bound['std'],2)

#判斷
bound = bound.set_index('spc_name')
df_isdefect = df_ctr.copy()
for spc_name in bound.index:
    upper = bound.loc['SPC_4']['upper']
    lower = bound.loc['SPC_4']['lower']
    df_isdefect[spc_name] = df_ctr[spc_name].map(lambda x: int(x>upper)|(x<lower) )
    
        
df_isdefect[np.concatenate((['timestamp','SPC_0'],bound.index))]    



,timestamp,SPC_0,SPC_4,SPC_5,SPC_10,SPC_11,SPC_7,SPC_6,SPC_2,SPC_39,...,SPC_57,SPC_29,SPC_30,SPC_31,SPC_32,SPC_33,SPC_34,SPC_35,SPC_36,SPC_37
0,1.569395e+09,156461,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1.569395e+09,156462,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1.569395e+09,156463,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1.569395e+09,156464,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1.569395e+09,156465,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,1.569395e+09,156466,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1.569395e+09,156467,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,1.569395e+09,156468,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,1.569395e+09,156469,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,1.569395e+09,156470,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
